In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [4]:
batch_size = 256
learning_rate = 0.0002
num_epoch = 10

In [5]:
mnist_train = dset.MNIST('./',train=True,transform=transforms.ToTensor(),
                         target_transform=None,download=True)
mnist_test = dset.MNIST('./',train=False,transform=transforms.ToTensor(),
                        target_transform=None,download=True)

#DataLoader : batch_size 만큼씩 넣어주고, 섞고 규칙에 따라 데이터 전달
train_loader = torch.utils.data.DataLoader(mnist_train,batch_size=batch_size,
                                           shuffle=True,num_workers=2, drop_last=True)
test_loader = torch.utils.data.DataLoader(mnist_test,batch_size=batch_size,
                                          shuffle=False, num_workers=2, drop_last=True)

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [8]:
class CNN(nn.Module):
  def __init__(self):
    #super는 CNN의 부모 class 인 nn.Module을 초기화
    super(CNN,self).__init__()
    self.layer = nn.Sequential(
        #kernel_size = 5X5 stride =1 padding= 0
        #input_shape =(batch,1,28,28)
        #image_size = floor((28 - 5 + 2*0)/1 +1)
        nn.Conv2d(1,16,5),
        nn.ReLU(),
        #input_shape = (batch,16,24,24) 24 ==  28-kernel+1
        nn.Conv2d(16,32,5),
        nn.ReLU(),
        #input_shape = (batch,32,20,20)
        nn.MaxPool2d(2,2),
        #input_shape = (batch, 32,10,10)
        nn.Conv2d(32,64,5),
        nn.ReLU(),
        #input_shape = (batch,64,6,6)
        nn.MaxPool2d(2,2)
        #output_shape = (batch,64,3,3)
    )

    self.fc_layer = nn.Sequential(
        nn.Linear(64*3*3,100),
        nn.ReLU(),
        nn.Linear(100,10)
    )

  def forward(self,x):
    #x input cnn
    out = self.layer(x).view(batch_size,-1)
    # dense 
    out = self.fc_layer(out)
    return out


In [9]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = CNN().to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)

In [10]:
loss_arr = []
for i in range(num_epoch):
  for j, [image,label] in enumerate(train_loader):
    #batch_size 만큼 gpu에 보냄
    x = image.to(device)
    y_ = label.to(device)

    optimizer.zero_grad()
    output = model.forward(x)
    loss = loss_func(output,y_)
    loss.backward()
    optimizer.step()

    if j % 1000 == 0:
      print(loss)
      #gpu에서 돌아가는 loss를 cpu파일로 변환 
      #detach()는 존 Tensor에서 gradient 전파가 안되는 텐서 생성 / 얕은 복사
      loss_arr.append(loss.cpu().detach().numpy())


tensor(2.3006, grad_fn=<NllLossBackward>)
tensor(0.3488, grad_fn=<NllLossBackward>)
tensor(0.0964, grad_fn=<NllLossBackward>)
tensor(0.0857, grad_fn=<NllLossBackward>)
tensor(0.0962, grad_fn=<NllLossBackward>)
tensor(0.1053, grad_fn=<NllLossBackward>)
tensor(0.0612, grad_fn=<NllLossBackward>)
tensor(0.0295, grad_fn=<NllLossBackward>)
tensor(0.0284, grad_fn=<NllLossBackward>)
tensor(0.0203, grad_fn=<NllLossBackward>)


In [ ]:
correct = 0
total = 0

#test data
with torch.no_grad():
  for image, label in test_loader:
    x = image.to(device)
    y_ = label.to(device)

    output = model.forward(x)
    #최댓값 인덱스
    _, output_index = torch.max(output,1)
    # 값만 뽑아내기 위해 (0)
    total += label.size(0)
    #배치에서 정답 개수를 더해줌
    correct += (output_index == y_).sum().float()

  print(f'accuracy of test data : {100*correct/total}')

AttributeError: ignored